记录各种Pytorch各种操作

1.Pytorch

In [2]:
# 冻结某层参数

# 模型参数转化为np.int8

In [ ]:
#toGpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bitmap.to(device)

#toCpu
bitmap = _bitmap.cpu().numpy()

#toCpu 去掉梯度
pred = pred.cpu().detach().numpy()

In [ ]:
## 保存模型与读取
# 保存整个网络
torch.save(net, PATH)
# 保存网络中的参数，速度快，占空间少
torch.save(net.state_dict(), PATH)
# 针对上面的保存方法，加载方法分别如下：
model_dict = torch.load(PATH)
model_dict = model.load_state_dict(torch.load(PATH))

# 试验中需保存更多参数，比如优化器的参数等
torch.save({'epoch': epochID + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN,
                            'optimizer': optimizer.state_dict(),'alpha': loss.alpha, 'gamma': loss.gamma},
                           checkpoint_path + '/m-' + launchTimestamp + '-' + str("%.4f" % lossMIN) + '.pth.tar')
# 对应的加载方式
def load_checkpoint(model, checkpoint_PATH, optimizer):
    if checkpoint != None:
        model_CKPT = torch.load(checkpoint_PATH)
        model.load_state_dict(model_CKPT['state_dict'])
        print('loading checkpoint!')
        optimizer.load_state_dict(model_CKPT['optimizer'])
    return model, optimizer
# 可能修改了一部分网络，比如加了一些，删除一些，等等，那么需要过滤这些参数，加载方式：
def load_checkpoint(model, checkpoint, optimizer, loadOptimizer):
    if checkpoint != 'No':
        print("loading checkpoint...")
        model_dict = model.state_dict()
        modelCheckpoint = torch.load(checkpoint)
        pretrained_dict = modelCheckpoint['state_dict']
        # 过滤操作
        new_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict.keys()} # 有些层不需要加载参数比如重新训练bilstm 
        model_dict.update(new_dict)
        # 打印出来，更新了多少的参数
        print('Total : {}, update: {}'.format(len(pretrained_dict), len(new_dict)))
        model.load_state_dict(model_dict)
        print("loaded finished!")
        # 如果不需要更新优化器那么设置为false
        if loadOptimizer == True:
            optimizer.load_state_dict(modelCheckpoint['optimizer'])
            print('loaded! optimizer')
        else:
            print('not loaded optimizer')
    else:
        print('No checkpoint is included')
    return model, optimizer

In [ ]:
# 冻结部分参数，训练另一部分参数
# 添加下面一句话到模型中
for p in self.parameters():
    p.requires_grad = False
# 比如加载了resnet预训练模型之后，在resnet的基础上连接了新的模块，resnet模块那部分可以暂时冻结不更新，只更新其他部分的参数，那么可以在下面加上那句话
class RESNET_MF(nn.Module):
    def __init__(self, model, pretrained):
        super(RESNET_MF, self).__init__()
        self.resnet = model(pretrained)
        for p in self.parameters():
            p.requires_grad = False
            self.f = SpectralNorm(nn.Conv2d(2048, 512, 1))
            self.g = SpectralNorm(nn.Conv2d(2048, 512, 1))
            self.h = SpectralNorm(nn.Conv2d(2048, 2048, 1))
# 同时在优化器中添加： filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), eps=1e-08, weight_decay=1e-5)

# 参数保存在有序的字典中，那么可以通过查找参数的名字对应的id值，进行冻结
model_dict = torch.load('net.pth.tar').state_dict()
dict_name = list(model_dict)
for i, p in enumerate(dict_name):
    print(i, p)
# 0 gamma
# 1 resnet.conv1.weight
# 2 resnet.bn1.weight
# 同样在模型中添加这样的代码
for i, p in enumerate(net.parameters()):
    if i < 165:
        p.requires_grad = False
# 同时在优化器中添加
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), eps=1e-08)

2. 整理dbnet代码，地址 https://github.com/WenmuZhou/DBNet.pytorch/tree/master/

In [1]:
# dbnet 填充区域
cv2.fillPoly(mask, box.reshape(1, -1, 2).astype(np.int32), 1)

# 找轮廓
contours, _ = cv2.findContours((bitmap * 255).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# 过滤轮廓
epsilon = 0.005 * cv2.arcLength(contour, True)

# 多边形拟合曲线
approx = cv2.approxPolyDP(contour, epsilon, True) 

# 向上，向下取整
np.floor(), np.ceil()

# 截取
np.clip(x.astype(np.int), 0, w)

NameError: name 'cv2' is not defined